<h1 align="center">IBM Applied Data Science Capstone</h1>
<h2 align="center">Battle of the Neighbourhoods</h2>
<h2 align="center">Paris and London</h2>

<img src="paris.jpg" width=400 height=600 /><img src="london.jpg" width=400 height=600 />

# 1. Introduction and Business Problem

The purpose of this project is to help tourists choose their destinations depending on what experiences the neighbourhoods have to offer and what the neighbourhoods would want to have. This also helps people make decisions if they are thinking about migrating to Paris or London or even if they want to relocate between neighbourhoods within the city.  

It also will help people to get awareness of the area and neighborhood before moving to a new city, state, country or place for their work or to start a new fresh life. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer.

### Work Flow
Using credentials of Foursquare API features of near-by places of the neighborhoods would be mined. Due to http request limitations the number of places per neighborhood parameter would reasonably be set to 100 and the radius parameter would be set to 500.

### Clustering Approach
To compare the similarities of two cities, we decided to explore neighborhoods, segment them, and group them into clusters to find similar neighborhoods in a big city. To be able to do that, we need to cluster data which is a form of unsupervised machine learning: k-means clustering algorithm


# 2. Data Description

We require geolocation data for both Paris and London. Postal codes in each city serve as a starting point. Using Postal codes we can find out the neighbourhoods, boroughs, venues and their most popular venue categories.

## Paris

To derive our solution, We scrape our data from https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e 

The JSON file has data about all the neighbourhoods in France, we limit it only Paris.

1. *postal_code* : Postal codes for France
2. *nom_comm* : Name of Neighbourhoods in France
3. *nom_dept* : Name of the boroughs, equivalent to towns in France
4. *geo_point_2d* : Tuple containing the latitude and longitude of the Neighbourhoods.

## London

To derive our solution, We scrape our data from https://en.wikipedia.org/wiki/List_of_areas_of_London

This wikipedia page has information about all the neighbourhoods, we limit it only London.

1. *borough* : Name of Neighbourhood
2. *town* : Name of borough
3. *post_code* : Postal codes for London.

This wikipedia page lacks information about the geographical locations. To solve this problem we use ArcGIS API

### ArcGIS API

ArcGIS Online enables us to connect people, locations, and data using interactive maps. Work with smart, data-driven styles and intuitive analysis tools that deliver location intelligence. Share your insights with the world or specific groups. We use ArcGIS to get the geo locations of the neighbourhoods of London. The following columns are added to our initial dataset which prepares our data. 

4. *latitude* : Latitude for Neighbourhood
5. *longitude* : Longitude for Neighbourhood
    
Based on all the information collected for both Paris and London, we have sufficient data to build our model. We cluster the neighbourhoods together based on similar venue categories. We then present our observations and findings. Using this data, our stakeholders can take the necessary decision.

## Foursquare API Data

We will need data about different venues in different neighbourhoods of that specific borough. In order to obtain information we use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighbourhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters. The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

1. *Neighbourhood* : Name of the Neighbourhood
2. *Neighbourhood Latitude* : Latitude of the Neighbourhood
3. *Neighbourhood Longitude* : Longitude of the Neighbourhood
4. *Venue* : Name of the Venue
5. *Venue Latitude* : Latitude of Venue
6. *Venue Longitude* : Longitude of Venue
7. *Venue Category* : Category of Venue

## Libraries Which are Used to Develope the Project:
Pandas: For creating and manipulating dataframes.

Folium: Python visualization library would be used to visualize the neighborhoods cluster distribution of using interactive leaflet map.

Scikit Learn: For importing k-means clustering.

Matplotlib: Python Plotting Module.

# 3. Methodology

We will be creating our model with the help of Python so we start off by importing all the required packages.

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# import k-means for the clustering stage
from sklearn.cluster import KMeans

import sys
!conda install --yes --prefix {sys.prefix} -c esri arcgis
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

Solving environment: ...working... done

# All requested packages already installed.



The approach taken here is to explore each of the cities individually, plot the map to show the neighbourhoods being considered and then build our model by clustering all of the similar neighbourhoods together and finally plot the new map with the clustered neighbourhoods. We draw insights and then compare and discuss our findings.

# Exploring Paris

### Data Collection

We read the json data with pandas.

In [2]:
paris_raw = pd.read_json('france-data.json')
paris_raw.head()

datasetid  \
0  correspondances-code-insee-code-postal   
1  correspondances-code-insee-code-postal   
2  correspondances-code-insee-code-postal   
3  correspondances-code-insee-code-postal   
4  correspondances-code-insee-code-postal   

                                   recordid  \
0  2bf36b38314b6c39dfbcd09225f97fa532b1fc45   
1  7ee82e74e059b443df18bb79fc5a19b1f05e5a88   
2  e2cd3186f07286705ed482a10b6aebd9de633c81   
3  868bf03527a1d0a9defe5cf4e6fa0a730d725699   
4  21e809b1d4480333c8b6fe7addd8f3b06f343e2c   

                                              fields  \
0  {'code_comm': '645', 'nom_dept': 'ESSONNE', 's...   
1  {'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...   
2  {'code_comm': '378', 'nom_dept': 'ESSONNE', 's...   
3  {'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...   
4  {'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...   

                                            geometry  \
0  {'type': 'Point', 'coordinates': [2.2517129721...   
1  {'type': 'Point', 'coordinates': [3.0529405055...   
2  {'type': 'Point', 'coordinates': [2.1971816504...   
3  {'type': 'Point', 'coordinates': [2.7097808131...   
4  {'type': 'Point', 'coordinates': [2.3335102498...   

                record_timestamp  
0  2016-09-21T00:29:06.175+02:00  
1  2016-09-21T00:29:06.175+02:00  
2  2016-09-21T00:29:06.175+02:00  
3  2016-09-21T00:29:06.175+02:00  
4  2016-09-21T00:29:06.175+02:00

### Data Preprocessing
We break down each of the nested fields and create the dataframe that we need

In [3]:
paris_field_data = pd.DataFrame()
for f in paris_raw.fields:
    dict_new = f
    paris_field_data = paris_field_data.append(dict_new, ignore_index=True)

paris_field_data.head()

code_arr code_cant code_comm code_dept code_reg  \
0        3        03       645        91       11   
1        3        20       133        77       11   
2        1        09       378        91       11   
3        5        14       243        77       11   
4        3        34       003        94       11   

                              geo_point_2d  \
0  [48.750443119964764, 2.251712972144151]   
1   [48.41256065214989, 3.052940505560729]   
2    [48.52726809075556, 2.19718165044305]   
3  [48.87307018579678, 2.7097808131278462]   
4   [48.80588035965699, 2.333510249842654]   

                                           geo_shape id_geofla insee_com  \
0  {'type': 'Polygon', 'coordinates': [[[2.238024...     16275     91645   
1  {'type': 'Polygon', 'coordinates': [[[3.076046...     31428     77133   
2  {'type': 'Polygon', 'coordinates': [[[2.203466...     30975     91378   
3  {'type': 'Polygon', 'coordinates': [[[2.727542...     17000     77243   
4  {'type': 'Polygon', 'coordinates': [[[2.343851...     32123     94003   

               nom_comm        nom_dept     nom_region  population  \
0  VERRIERES-LE-BUISSON         ESSONNE  ILE-DE-FRANCE        15.5   
1  COURCELLES-EN-BASSEE  SEINE-ET-MARNE  ILE-DE-FRANCE         0.2   
2             MAUCHAMPS         ESSONNE  ILE-DE-FRANCE         0.3   
3       LAGNY-SUR-MARNE  SEINE-ET-MARNE  ILE-DE-FRANCE        20.2   
4               ARCUEIL    VAL-DE-MARNE  ILE-DE-FRANCE        19.5   

  postal_code            statut  superficie  z_moyen  
0       91370    Commune simple       999.0    121.0  
1       77126    Commune simple      1082.0     88.0  
2       91730    Commune simple       313.0    150.0  
3       77400  Chef-lieu canton       579.0     71.0  
4       94110  Chef-lieu canton       232.0     70.0

### Feature Selection
We take the columns that we require, in case of paris it would be the geo_point_2d, nom_dept, nom_comm and postal_code

In [4]:
df_paris = paris_field_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df_paris

postal_code              nom_comm           nom_dept  \
0          91370  VERRIERES-LE-BUISSON            ESSONNE   
1          77126  COURCELLES-EN-BASSEE     SEINE-ET-MARNE   
2          91730             MAUCHAMPS            ESSONNE   
3          77400       LAGNY-SUR-MARNE     SEINE-ET-MARNE   
4          94110               ARCUEIL       VAL-DE-MARNE   
...          ...                   ...                ...   
1295       77520     CESSOY-EN-MONTOIS     SEINE-ET-MARNE   
1296       93420            VILLEPINTE  SEINE-SAINT-DENIS   
1297       77130         CANNES-ECLUSE     SEINE-ET-MARNE   
1298       78930              VILLETTE           YVELINES   
1299       95270  LE PLESSIS-LUZARCHES         VAL-D'OISE   

                                 geo_point_2d  
0     [48.750443119964764, 2.251712972144151]  
1      [48.41256065214989, 3.052940505560729]  
2       [48.52726809075556, 2.19718165044305]  
3     [48.87307018579678, 2.7097808131278462]  
4      [48.80588035965699, 2.333510249842654]  
...                                       ...  
1295   [48.50730730461658, 3.138844194183689]  
1296   [48.95902025378707, 2.536306342059409]  
1297   [48.36403767307805, 2.990786679832767]  
1298  [48.92627887061508, 1.6937417245662671]  
1299  [49.09572967201378, 2.4547564431234923]  

[1300 rows x 4 columns]

### Feature Engineering
We localize to only include Paris

In [5]:
df_paris = df_paris[df_paris['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris

postal_code                  nom_comm nom_dept  \
0        75009   PARIS-9E-ARRONDISSEMENT    PARIS   
1        75002   PARIS-2E-ARRONDISSEMENT    PARIS   
2        75011  PARIS-11E-ARRONDISSEMENT    PARIS   
3        75003   PARIS-3E-ARRONDISSEMENT    PARIS   
4        75006   PARIS-6E-ARRONDISSEMENT    PARIS   
5        75004   PARIS-4E-ARRONDISSEMENT    PARIS   
6        75001  PARIS-1ER-ARRONDISSEMENT    PARIS   
7        75017  PARIS-17E-ARRONDISSEMENT    PARIS   
8        75008   PARIS-8E-ARRONDISSEMENT    PARIS   
9        75013  PARIS-13E-ARRONDISSEMENT    PARIS   
10       75012  PARIS-12E-ARRONDISSEMENT    PARIS   
11       75005   PARIS-5E-ARRONDISSEMENT    PARIS   
12       75019  PARIS-19E-ARRONDISSEMENT    PARIS   
13       75020  PARIS-20E-ARRONDISSEMENT    PARIS   
14       75010  PARIS-10E-ARRONDISSEMENT    PARIS   
15       75016  PARIS-16E-ARRONDISSEMENT    PARIS   
16       75018  PARIS-18E-ARRONDISSEMENT    PARIS   
17       75007   PARIS-7E-ARRONDISSEMENT    PARIS   
18       75015  PARIS-15E-ARRONDISSEMENT    PARIS   
19       75014  PARIS-14E-ARRONDISSEMENT    PARIS   

                               geo_point_2d  
0    [48.87689616237872, 2.337460241388529]  
1    [48.86790337886785, 2.344107166658533]  
2    [48.85941549762748, 2.378741060237548]  
3    [48.86305413181178, 2.359361058970589]  
4    [48.84896809191946, 2.332670898588416]  
5   [48.854228281954754, 2.357361938142205]  
6     [48.8626304851685, 2.336293446550539]  
7    [48.88733716648682, 2.307485559493426]  
8    [48.87252726662346, 2.312582560420059]  
9    [48.82871768452136, 2.362468228516128]  
10   [48.83515623066034, 2.419807034965275]  
11  [48.844508659617546, 2.349859385560182]  
12   [48.88686862295828, 2.384694327870042]  
13   [48.86318677744551, 2.400819826729021]  
14   [48.87602855694339, 2.361112904561707]  
15   [48.86039876035177, 2.262099559395783]  
16  [48.892735074561706, 2.348711933867703]  
17   [48.85608259819694, 2.312438687733857]  
18   [48.84015541860987, 2.293559372435076]  
19   [48.82899321160942, 2.327100883257538]

In [6]:
df_paris.shape

(20, 4)

### Gelocations of the Neighbourhoods of Paris
We don't need to get the geo coordinates using an external data source or collect it with the arcgis API call since we already have it stored in the *geo_point_2d* column as a tuple in the *df_paris* dataframe.

In [7]:
temp1 = df_paris['geo_point_2d']
temp1

0      [48.87689616237872, 2.337460241388529]
1      [48.86790337886785, 2.344107166658533]
2      [48.85941549762748, 2.378741060237548]
3      [48.86305413181178, 2.359361058970589]
4      [48.84896809191946, 2.332670898588416]
5     [48.854228281954754, 2.357361938142205]
6       [48.8626304851685, 2.336293446550539]
7      [48.88733716648682, 2.307485559493426]
8      [48.87252726662346, 2.312582560420059]
9      [48.82871768452136, 2.362468228516128]
10     [48.83515623066034, 2.419807034965275]
11    [48.844508659617546, 2.349859385560182]
12     [48.88686862295828, 2.384694327870042]
13     [48.86318677744551, 2.400819826729021]
14     [48.87602855694339, 2.361112904561707]
15     [48.86039876035177, 2.262099559395783]
16    [48.892735074561706, 2.348711933867703]
17     [48.85608259819694, 2.312438687733857]
18     [48.84015541860987, 2.293559372435076]
19     [48.82899321160942, 2.327100883257538]
Name: geo_point_2d, dtype: object

In [8]:
paris_latlng = df_paris['geo_point_2d'].astype('str')

Spliting the geo_point_2d column into latitude and longitude.

#### Latitude

In [9]:
paris_lat = paris_latlng.apply(lambda x: x.split(',')[0])
paris_lat = paris_lat.apply(lambda x: x.lstrip('['))
paris_lat

0      48.87689616237872
1      48.86790337886785
2      48.85941549762748
3      48.86305413181178
4      48.84896809191946
5     48.854228281954754
6       48.8626304851685
7      48.88733716648682
8      48.87252726662346
9      48.82871768452136
10     48.83515623066034
11    48.844508659617546
12     48.88686862295828
13     48.86318677744551
14     48.87602855694339
15     48.86039876035177
16    48.892735074561706
17     48.85608259819694
18     48.84015541860987
19     48.82899321160942
Name: geo_point_2d, dtype: object

#### Longitude

In [10]:
paris_lng = paris_latlng.apply(lambda x: x.split(',')[1])
paris_lng = paris_lng.apply(lambda x: x.rstrip(']'))
paris_lng

0      2.337460241388529
1      2.344107166658533
2      2.378741060237548
3      2.359361058970589
4      2.332670898588416
5      2.357361938142205
6      2.336293446550539
7      2.307485559493426
8      2.312582560420059
9      2.362468228516128
10     2.419807034965275
11     2.349859385560182
12     2.384694327870042
13     2.400819826729021
14     2.361112904561707
15     2.262099559395783
16     2.348711933867703
17     2.312438687733857
18     2.293559372435076
19     2.327100883257538
Name: geo_point_2d, dtype: object

In [11]:
paris_geo_lat  = pd.DataFrame(paris_lat.astype(float))
paris_geo_lat.columns=['Latitude']
paris_geo_lng = pd.DataFrame(paris_lng.astype(float))
paris_geo_lng.columns=['Longitude']

In [12]:
paris_combined_data = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_geo_lat, paris_geo_lng], axis=1)
paris_combined_data

postal_code                  nom_comm nom_dept   Latitude  Longitude
0        75009   PARIS-9E-ARRONDISSEMENT    PARIS  48.876896   2.337460
1        75002   PARIS-2E-ARRONDISSEMENT    PARIS  48.867903   2.344107
2        75011  PARIS-11E-ARRONDISSEMENT    PARIS  48.859415   2.378741
3        75003   PARIS-3E-ARRONDISSEMENT    PARIS  48.863054   2.359361
4        75006   PARIS-6E-ARRONDISSEMENT    PARIS  48.848968   2.332671
5        75004   PARIS-4E-ARRONDISSEMENT    PARIS  48.854228   2.357362
6        75001  PARIS-1ER-ARRONDISSEMENT    PARIS  48.862630   2.336293
7        75017  PARIS-17E-ARRONDISSEMENT    PARIS  48.887337   2.307486
8        75008   PARIS-8E-ARRONDISSEMENT    PARIS  48.872527   2.312583
9        75013  PARIS-13E-ARRONDISSEMENT    PARIS  48.828718   2.362468
10       75012  PARIS-12E-ARRONDISSEMENT    PARIS  48.835156   2.419807
11       75005   PARIS-5E-ARRONDISSEMENT    PARIS  48.844509   2.349859
12       75019  PARIS-19E-ARRONDISSEMENT    PARIS  48.886869   2.384694
13       75020  PARIS-20E-ARRONDISSEMENT    PARIS  48.863187   2.400820
14       75010  PARIS-10E-ARRONDISSEMENT    PARIS  48.876029   2.361113
15       75016  PARIS-16E-ARRONDISSEMENT    PARIS  48.860399   2.262100
16       75018  PARIS-18E-ARRONDISSEMENT    PARIS  48.892735   2.348712
17       75007   PARIS-7E-ARRONDISSEMENT    PARIS  48.856083   2.312439
18       75015  PARIS-15E-ARRONDISSEMENT    PARIS  48.840155   2.293559
19       75014  PARIS-14E-ARRONDISSEMENT    PARIS  48.828993   2.327101

### Co-ordinates for Paris

In [13]:
paris = geocode(address='Paris, France, FR')[0]
paris_lng_coords = paris['location']['x']
paris_lat_coords = paris['location']['y']
print("The geolocation of Paris: ", paris_lat_coords, paris_lng_coords)

The geolocation of Paris:  48.85341000000005 2.3488000000000397


### Visualize the Map of Paris

In [14]:
# Creating the map of Paris
map_Paris= folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_combined_data['Latitude'], paris_combined_data['Longitude'], paris_combined_data['nom_comm'], paris_combined_data['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

### Venues in Paris
To proceed with the next part, we need to define Foursquare API credentials.
Using Foursquare API, we are able to get the venue and venue categories around each neighbourhood in Paris.

In [15]:
CLIENT_ID = 'MW0HNMAAVZ2AHN2GJQVRURBGN5HEORAR0EVEYMKI10WYYNNM'
CLIENT_SECRET = 'DQ1ZEWFMCBKBC0311AJQE1T3HNSTQRP1RDVWOFZCVYLS20DB'
VERSION = '20210113' # Foursquare API version

In [16]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
venues_in_Paris = getNearbyVenues(paris_combined_data['nom_comm'], 
                                  paris_combined_data['Latitude'], 
                                  paris_combined_data['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [18]:
venues_in_Paris.head()

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0  PARIS-9E-ARRONDISSEMENT               48.876896                  2.33746   
1  PARIS-9E-ARRONDISSEMENT               48.876896                  2.33746   
2  PARIS-9E-ARRONDISSEMENT               48.876896                  2.33746   
3  PARIS-9E-ARRONDISSEMENT               48.876896                  2.33746   
4  PARIS-9E-ARRONDISSEMENT               48.876896                  2.33746   

                    Venue                 Venue Category  
0                  So Nat  Vegetarian / Vegan Restaurant  
1                     RAP                   Gourmet Shop  
2              Farine & O                         Bakery  
3  Le Bouclier de Bacchus                       Wine Bar  
4     Place Saint-Georges                          Plaza

### Grouping by Venue Categories
We need to now see how many Venue Categories are there for further processing

In [19]:
venues_in_Paris.groupby('Venue Category').max()

Neighbourhood  Neighbourhood Latitude  \
Venue Category                                                             
Afghan Restaurant       PARIS-11E-ARRONDISSEMENT               48.859415   
African Restaurant       PARIS-9E-ARRONDISSEMENT               48.876896   
American Restaurant     PARIS-19E-ARRONDISSEMENT               48.892735   
Antique Shop             PARIS-9E-ARRONDISSEMENT               48.876896   
Argentinian Restaurant   PARIS-3E-ARRONDISSEMENT               48.863054   
...                                          ...                     ...   
Wine Bar                 PARIS-9E-ARRONDISSEMENT               48.892735   
Wine Shop                PARIS-3E-ARRONDISSEMENT               48.892735   
Women's Store            PARIS-2E-ARRONDISSEMENT               48.867903   
Zoo                     PARIS-12E-ARRONDISSEMENT               48.835156   
Zoo Exhibit             PARIS-12E-ARRONDISSEMENT               48.835156   

                        Neighbourhood Longitude  \
Venue Category                                    
Afghan Restaurant                      2.378741   
African Restaurant                     2.361113   
American Restaurant                    2.384694   
Antique Shop                           2.337460   
Argentinian Restaurant                 2.359361   
...                                         ...   
Wine Bar                               2.400820   
Wine Shop                              2.361113   
Women's Store                          2.344107   
Zoo                                    2.419807   
Zoo Exhibit                            2.419807   

                                                           Venue  
Venue Category                                                    
Afghan Restaurant                                    Afghanistan  
African Restaurant                             Wally Le Saharien  
American Restaurant                                     Harper's  
Antique Shop                             Hôtel des Ventes Drouot  
Argentinian Restaurant                                     Anahi  
...                                                          ...  
Wine Bar                                               Ô Château  
Wine Shop                                         Trois Fois Vin  
Women's Store                               L'Appartement Sézane  
Zoo                                     Parc zoologique de Paris  
Zoo Exhibit             Grande Serre du Parc Zoologique de Paris  

[204 rows x 4 columns]

### One Hot Encoding 
We need to encode our venue categories to get a better result for our clustering

In [20]:
Paris_venue_cat = pd.get_dummies(venues_in_Paris[['Venue Category']], prefix="", prefix_sep="")
Paris_venue_cat

Afghan Restaurant  African Restaurant  American Restaurant  \
0                     0                   0                    0   
1                     0                   0                    0   
2                     0                   0                    0   
3                     0                   0                    0   
4                     0                   0                    0   
...                 ...                 ...                  ...   
1282                  0                   0                    0   
1283                  0                   0                    0   
1284                  0                   0                    0   
1285                  0                   0                    0   
1286                  0                   0                    0   

      Antique Shop  Argentinian Restaurant  Art Gallery  Art Museum  \
0                0                       0            0           0   
1                0                       0            0           0   
2                0                       0            0           0   
3                0                       0            0           0   
4                0                       0            0           0   
...            ...                     ...          ...         ...   
1282             0                       0            0           0   
1283             0                       0            0           0   
1284             0                       0            0           0   
1285             0                       0            0           0   
1286             0                       0            0           0   

      Arts & Crafts Store  Asian Restaurant  Baby Store  ...  \
0                       0                 0           0  ...   
1                       0                 0           0  ...   
2                       0                 0           0  ...   
3                       0                 0           0  ...   
4                       0                 0           0  ...   
...                   ...               ...         ...  ...   
1282                    0                 0           0  ...   
1283                    0                 0           0  ...   
1284                    0                 0           0  ...   
1285                    0                 0           0  ...   
1286                    0                 0           0  ...   

      Vegetarian / Vegan Restaurant  Venezuelan Restaurant  Video Game Store  \
0                                 1                      0                 0   
1                                 0                      0                 0   
2                                 0                      0                 0   
3                                 0                      0                 0   
4                                 0                      0                 0   
...                             ...                    ...               ...   
1282                              0                      0                 0   
1283                              0                      0                 0   
1284                              0                      0                 0   
1285                              0                      0                 0   
1286                              0                      0                 0   

      Vietnamese Restaurant  Waterfront  Wine Bar  Wine Shop  Women's Store  \
0                         0           0         0          0              0   
1                         0           0         0          0              0   
2                         0           0         0          0              0   
3                         0           0         1          0              0   
4                         0           0         0          0              0   
...                     ...         ...       ...        ...            ...   
1282                      0           0         0    

In [21]:
Paris_venue_cat['Neighbourhood'] = venues_in_Paris['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [Paris_venue_cat.columns[-1]] + list(Paris_venue_cat.columns[:-1])
Paris_venue_cat = Paris_venue_cat[fixed_columns]

Paris_venue_cat.head()

Neighbourhood  Afghan Restaurant  African Restaurant  \
0  PARIS-9E-ARRONDISSEMENT                  0                   0   
1  PARIS-9E-ARRONDISSEMENT                  0                   0   
2  PARIS-9E-ARRONDISSEMENT                  0                   0   
3  PARIS-9E-ARRONDISSEMENT                  0                   0   
4  PARIS-9E-ARRONDISSEMENT                  0                   0   

   American Restaurant  Antique Shop  Argentinian Restaurant  Art Gallery  \
0                    0             0                       0            0   
1                    0             0                       0            0   
2                    0             0                       0            0   
3                    0             0                       0            0   
4                    0             0                       0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  ...  \
0           0                    0                 0  ...   
1           0                    0                 0  ...   
2           0                    0                 0  ...   
3           0                    0                 0  ...   
4           0                    0                 0  ...   

   Vegetarian / Vegan Restaurant  Venezuelan Restaurant  Video Game Store  \
0                              1                      0                 0   
1                              0                      0                 0   
2                              0                      0                 0   
3                              0                      0                 0   
4                              0                      0                 0   

   Vietnamese Restaurant  Waterfront  Wine Bar  Wine Shop  Women's Store  Zoo  \
0                      0           0         0          0              0    0   
1                      0           0         0          0              0    0   
2                      0           0         0          0              0    0   
3                      0           0         1          0              0    0   
4                      0           0         0          0              0    0   

   Zoo Exhibit  
0            0  
1            0  
2            0  
3            0  
4            0  

[5 rows x 205 columns]

### Venue categories mean value
We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [22]:
Paris_grouped = Paris_venue_cat.groupby('Neighbourhood').mean().reset_index()
Paris_grouped.head()

Neighbourhood  Afghan Restaurant  African Restaurant  \
0  PARIS-10E-ARRONDISSEMENT            0.00000                0.02   
1  PARIS-11E-ARRONDISSEMENT            0.02439                0.00   
2  PARIS-12E-ARRONDISSEMENT            0.00000                0.00   
3  PARIS-13E-ARRONDISSEMENT            0.00000                0.00   
4  PARIS-14E-ARRONDISSEMENT            0.00000                0.00   

   American Restaurant  Antique Shop  Argentinian Restaurant  Art Gallery  \
0                  0.0           0.0                     0.0          0.0   
1                  0.0           0.0                     0.0          0.0   
2                  0.0           0.0                     0.0          0.0   
3                  0.0           0.0                     0.0          0.0   
4                  0.0           0.0                     0.0          0.0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  ...  \
0     0.00000                  0.0          0.030000  ...   
1     0.02439                  0.0          0.024390  ...   
2     0.00000                  0.0          0.000000  ...   
3     0.00000                  0.0          0.215385  ...   
4     0.00000                  0.0          0.000000  ...   

   Vegetarian / Vegan Restaurant  Venezuelan Restaurant  Video Game Store  \
0                           0.01                    0.0           0.00000   
1                           0.00                    0.0           0.02439   
2                           0.00                    0.0           0.00000   
3                           0.00                    0.0           0.00000   
4                           0.00                    0.0           0.00000   

   Vietnamese Restaurant  Waterfront  Wine Bar  Wine Shop  Women's Store  Zoo  \
0               0.000000         0.0   0.02000       0.02            0.0  0.0   
1               0.073171         0.0   0.02439       0.00            0.0  0.0   
2               0.000000         0.0   0.00000       0.00            0.0  0.2   
3               0.200000         0.0   0.00000       0.00            0.0  0.0   
4               0.000000         0.0   0.00000       0.00            0.0  0.0   

   Zoo Exhibit  
0          0.0  
1          0.0  
2          0.2  
3          0.0  
4          0.0  

[5 rows x 205 columns]

### Top venue categories
Using our defined function to get the top venue categories in the neighbourhoods of Paris.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

There are way too many venue categories, we can take the top 10 to cluster the neighbourhoods.
Creating a function to label the columns of the venue correctly

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [25]:
# create a new dataframe for Paris
neighborhoods_venues_sorted_paris = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_paris['Neighbourhood'] = Paris_grouped['Neighbourhood']

for ind in np.arange(Paris_grouped.shape[0]):
    neighborhoods_venues_sorted_paris.iloc[ind, 1:] = return_most_common_venues(Paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_paris.head()

Neighbourhood 1st Most Common Venue  2nd Most Common Venue  \
0  PARIS-10E-ARRONDISSEMENT     French Restaurant                  Hotel   
1  PARIS-11E-ARRONDISSEMENT            Restaurant  Vietnamese Restaurant   
2  PARIS-12E-ARRONDISSEMENT           Zoo Exhibit                 Bistro   
3  PARIS-13E-ARRONDISSEMENT      Asian Restaurant  Vietnamese Restaurant   
4  PARIS-14E-ARRONDISSEMENT     French Restaurant                  Hotel   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                Bistro                  Café           Coffee Shop   
1     French Restaurant                Bakery                  Café   
2   Monument / Landmark           Supermarket                   Zoo   
3     French Restaurant    Chinese Restaurant       Thai Restaurant   
4                Bistro   Japanese Restaurant              Tea Room   

    6th Most Common Venue 7th Most Common Venue   8th Most Common Venue  \
0             Pizza Place     Indian Restaurant      Italian Restaurant   
1      Italian Restaurant           Pastry Shop                     Bar   
2  Argentinian Restaurant                Garden  Furniture / Home Store   
3                   Hotel             Juice Bar            Dessert Shop   
4             Pizza Place             Pet Store                    Café   

  9th Most Common Venue 10th Most Common Venue  
0      Asian Restaurant    Japanese Restaurant  
1             Gastropub     Mexican Restaurant  
2    Frozen Yogurt Shop      French Restaurant  
3        Sandwich Place            Coffee Shop  
4  Fast Food Restaurant      Food & Drink Shop

## Model Building

### K Means
Let's cluster the city of Paris to roughly 5 to make it easier to analyze. We use the K Means clustering technique to do so.

In [26]:
# set number of clusters
k_num_clusters = 5

Paris_grouped_clustering = Paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_Paris = KMeans(n_clusters=k_num_clusters, random_state=0).fit(Paris_grouped_clustering)
kmeans_Paris

KMeans(n_clusters=5, random_state=0)

### Labelling Clustered Data

In [27]:
kmeans_Paris.labels_

array([0, 0, 2, 3, 4, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0])

In [28]:
neighborhoods_venues_sorted_paris.insert(0, 'Cluster Labels', kmeans_Paris.labels_ +1)

In [29]:
paris_data = paris_combined_data
paris_data = paris_data.join(neighborhoods_venues_sorted_paris.set_index('Neighbourhood'), on='nom_comm')
paris_data.head()

postal_code                  nom_comm nom_dept   Latitude  Longitude  \
0       75009   PARIS-9E-ARRONDISSEMENT    PARIS  48.876896   2.337460   
1       75002   PARIS-2E-ARRONDISSEMENT    PARIS  48.867903   2.344107   
2       75011  PARIS-11E-ARRONDISSEMENT    PARIS  48.859415   2.378741   
3       75003   PARIS-3E-ARRONDISSEMENT    PARIS  48.863054   2.359361   
4       75006   PARIS-6E-ARRONDISSEMENT    PARIS  48.848968   2.332671   

   Cluster Labels 1st Most Common Venue  2nd Most Common Venue  \
0               1     French Restaurant                  Hotel   
1               1     French Restaurant           Cocktail Bar   
2               1            Restaurant  Vietnamese Restaurant   
3               1     French Restaurant    Japanese Restaurant   
4               1           Pastry Shop         Chocolate Shop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0   Japanese Restaurant                Bistro                Bakery   
1           Coffee Shop              Wine Bar                Bakery   
2     French Restaurant                Bakery                  Café   
3                Bakery    Italian Restaurant          Gourmet Shop   
4     French Restaurant                Bakery              Wine Bar   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                Lounge              Wine Bar          Cocktail Bar   
1                 Hotel              Creperie    Italian Restaurant   
2    Italian Restaurant           Pastry Shop                   Bar   
3           Coffee Shop          Cocktail Bar           Art Gallery   
4                 Plaza              Fountain              Tea Room   

  9th Most Common Venue 10th Most Common Venue  
0            Restaurant                   Café  
1       Thai Restaurant         Sandwich Place  
2             Gastropub     Mexican Restaurant  
3        Sandwich Place               Wine Bar  
4        Sandwich Place    Lebanese Restaurant

In [30]:
paris_data_nonan = paris_data.dropna(subset=['Cluster Labels'])

### Visualizing the clustered neighbourhood
Let's plot the clusters

In [31]:
map_clusters_paris = folium.Map(location=[paris_lat_coords, paris_lng_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_data_nonan['Latitude'], paris_data_nonan['Longitude'], paris_data_nonan['nom_comm'], paris_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + ' ' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_paris)
        
map_clusters_paris

### Examining our Clusters

Cluster 1

In [32]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 1,
                     paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

nom_comm  Cluster Labels 1st Most Common Venue  \
0    PARIS-9E-ARRONDISSEMENT               1     French Restaurant   
1    PARIS-2E-ARRONDISSEMENT               1     French Restaurant   
2   PARIS-11E-ARRONDISSEMENT               1            Restaurant   
3    PARIS-3E-ARRONDISSEMENT               1     French Restaurant   
4    PARIS-6E-ARRONDISSEMENT               1           Pastry Shop   
5    PARIS-4E-ARRONDISSEMENT               1     French Restaurant   
6   PARIS-1ER-ARRONDISSEMENT               1   Japanese Restaurant   
11   PARIS-5E-ARRONDISSEMENT               1     French Restaurant   
12  PARIS-19E-ARRONDISSEMENT               1     French Restaurant   
13  PARIS-20E-ARRONDISSEMENT               1                Bakery   
14  PARIS-10E-ARRONDISSEMENT               1     French Restaurant   
16  PARIS-18E-ARRONDISSEMENT               1     French Restaurant   
18  PARIS-15E-ARRONDISSEMENT               1                 Hotel   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Hotel   Japanese Restaurant                Bistro   
1            Cocktail Bar           Coffee Shop              Wine Bar   
2   Vietnamese Restaurant     French Restaurant                Bakery   
3     Japanese Restaurant                Bakery    Italian Restaurant   
4          Chocolate Shop     French Restaurant                Bakery   
5          Ice Cream Shop                  Park        Clothing Store   
6       French Restaurant                 Plaza                 Hotel   
11                  Hotel    Italian Restaurant                 Plaza   
12                    Bar                 Hotel           Supermarket   
13                  Plaza                Bistro   Japanese Restaurant   
14                  Hotel                Bistro                  Café   
16                    Bar    Italian Restaurant                 Hotel   
18     Italian Restaurant     French Restaurant     Indian Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                 Bakery                Lounge              Wine Bar   
1                 Bakery                 Hotel              Creperie   
2                   Café    Italian Restaurant           Pastry Shop   
3           Gourmet Shop           Coffee Shop          Cocktail Bar   
4               Wine Bar                 Plaza              Fountain   
5                  Hotel              Wine Bar           Pastry Shop   
6     Italian Restaurant                  Café            Art Museum   
11                Bakery           Coffee Shop                   Pub   
12                Bistro    Seafood Restaurant              Beer Bar   
13     French Restaurant           Supermarket                   Bar   
14           Coffee Shop           Pizza Place     Indian Restaurant   
16                  Café            Restaurant                 Plaza   
18   Japanese Restaurant       Thai Restaurant            Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Cocktail Bar            Restaurant                   Café  
1     Italian Restaurant       Thai Restaurant         Sandwich Place  
2                    Bar             Gastropub     Mexican Restaurant  
3            Art Gallery        Sandwich Place               Wine Bar  
4               Tea Room        Sandwich Place    Lebanese Restaurant  
5       Pedestrian Plaza                 Plaza                 Bistro  
6                 Bakery                Garden      Korean Restaurant  
11                  Café        Science Museum                    Bar  
12               Brewery          Burger Joint                   Café  
13    Italian Restaurant     Food & Drink Shop                   Café  
14    Italian Restaurant      Asian Restaurant    Japanese Restaurant  
16           Pizza Place                Bistro  Vietnamese Restaurant  
18   Lebanese Restaurant             Brasserie                   Park

Cluster 2

In [33]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 2, 
                     paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

nom_comm  Cluster Labels 1st Most Common Venue  \
15  PARIS-16E-ARRONDISSEMENT               2                 Plaza   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
15                  Lake                  Park     French Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
15           Bus Station            Art Museum         Boat or Ferry   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
15  Bike Rental / Bike Share           Zoo Exhibit               Fountain

Cluster 3

In [34]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 3, 
                     paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

nom_comm  Cluster Labels 1st Most Common Venue  \
10  PARIS-12E-ARRONDISSEMENT               3           Zoo Exhibit   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10                Bistro   Monument / Landmark           Supermarket   

   5th Most Common Venue   6th Most Common Venue 7th Most Common Venue  \
10                   Zoo  Argentinian Restaurant                Garden   

     8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
10  Furniture / Home Store    Frozen Yogurt Shop      French Restaurant

Cluster 4

In [35]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 4, 
                     paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

nom_comm  Cluster Labels 1st Most Common Venue  \
9  PARIS-13E-ARRONDISSEMENT               4      Asian Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
9  Vietnamese Restaurant     French Restaurant    Chinese Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
9       Thai Restaurant                 Hotel             Juice Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
9          Dessert Shop        Sandwich Place            Coffee Shop

Cluster 5

In [36]:
paris_data_nonan.loc[paris_data_nonan['Cluster Labels'] == 5, 
                     paris_data_nonan.columns[[1] + list(range(5, paris_data_nonan.shape[1]))]]

nom_comm  Cluster Labels 1st Most Common Venue  \
7   PARIS-17E-ARRONDISSEMENT               5     French Restaurant   
8    PARIS-8E-ARRONDISSEMENT               5     French Restaurant   
17   PARIS-7E-ARRONDISSEMENT               5     French Restaurant   
19  PARIS-14E-ARRONDISSEMENT               5     French Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
7                  Hotel    Italian Restaurant                Bakery   
8                  Hotel                   Spa          Cocktail Bar   
17                 Hotel                  Café    Italian Restaurant   
19                 Hotel                Bistro   Japanese Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
7                   Café                Bistro            Restaurant   
8          Grocery Store      Department Store   Corsican Restaurant   
17                 Plaza        History Museum          Cocktail Bar   
19              Tea Room           Pizza Place             Pet Store   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
7    Japanese Restaurant              Bus Stop                  Plaza  
8                 Resort                  Park   Fast Food Restaurant  
17   Japanese Restaurant                Bistro              Irish Pub  
19                  Café  Fast Food Restaurant      Food & Drink Shop

----------------------------------------
--------------------------------------

# Exploring London

### Data Collection
To get the neighbourhoods in london, we start by scraping the list of areas of london wiki page.

In [37]:
url_london = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_london_url = requests.get(url_london)
wiki_london_url

<Response [200]>

In [38]:
wiki_london_data = pd.read_html(wiki_london_url.text)
wiki_london_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 526         Woolwich                          Greenwich          LONDON   
 527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 529          Yeading                         Hillingdon           HAYES   
 

In [39]:
wiki_london_data = wiki_london_data[1]
wiki_london_data

Location                     London borough       Post town  \
0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
2          Addington                         Croydon[8]         CROYDON   
3         Addiscombe                         Croydon[8]         CROYDON   
4        Albany Park                             Bexley  BEXLEY, SIDCUP   
..               ...                                ...             ...   
526         Woolwich                          Greenwich          LONDON   
527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
529          Yeading                         Hillingdon           HAYES   
530         Yiewsley                         Hillingdon    WEST DRAYTON   

    Postcode district Dial code OS grid ref  
0                 SE2       020    TQ465785  
1              W3, W4       020    TQ205805  
2                 CR0       020    TQ375645  
3                 CR0       020    TQ345665  
4           DA5, DA14       020    TQ478728  
..                ...       ...         ...  
526              SE18       020    TQ435795  
527               KT4       020    TQ225655  
528               W12       020    TQ225815  
529               UB4       020    TQ115825  
530               UB7       020    TQ063804  

[531 rows x 6 columns]

### Data Preprocessing
we remove the spaces in the column titles and then we add _ between words.

In [40]:
wiki_london_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_london_data

Location                     London borough       Post_town  \
0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
2          Addington                         Croydon[8]         CROYDON   
3         Addiscombe                         Croydon[8]         CROYDON   
4        Albany Park                             Bexley  BEXLEY, SIDCUP   
..               ...                                ...             ...   
526         Woolwich                          Greenwich          LONDON   
527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
529          Yeading                         Hillingdon           HAYES   
530         Yiewsley                         Hillingdon    WEST DRAYTON   

    Postcode district Dial code OS_grid_ref  
0                 SE2       020    TQ465785  
1              W3, W4       020    TQ205805  
2                 CR0       020    TQ375645  
3                 CR0       020    TQ345665  
4           DA5, DA14       020    TQ478728  
..                ...       ...         ...  
526              SE18       020    TQ435795  
527               KT4       020    TQ225655  
528               W12       020    TQ225815  
529               UB4       020    TQ115825  
530               UB7       020    TQ063804  

[531 rows x 6 columns]

### Feature Selection
We need only the boroughs, postal codes, post town for further steps. We can drop the locations, dial codes and OS grid.

In [41]:
df_london = wiki_london_data.drop( [ wiki_london_data.columns[0], 
                              wiki_london_data.columns[4], 
                              wiki_london_data.columns[5] ], axis=1)
df_london.head()

London borough       Post_town Postcode district
0              Bexley, Greenwich [7]          LONDON               SE2
1  Ealing, Hammersmith and Fulham[8]          LONDON            W3, W4
2                         Croydon[8]         CROYDON               CR0
3                         Croydon[8]         CROYDON               CR0
4                             Bexley  BEXLEY, SIDCUP         DA5, DA14

In [42]:
df_london.columns = ['borough','town','post_code']
df_london

borough            town  post_code
0                Bexley, Greenwich [7]          LONDON        SE2
1    Ealing, Hammersmith and Fulham[8]          LONDON     W3, W4
2                           Croydon[8]         CROYDON        CR0
3                           Croydon[8]         CROYDON        CR0
4                               Bexley  BEXLEY, SIDCUP  DA5, DA14
..                                 ...             ...        ...
526                          Greenwich          LONDON       SE18
527       Sutton, Kingston upon Thames  WORCESTER PARK        KT4
528             Hammersmith and Fulham          LONDON        W12
529                         Hillingdon           HAYES        UB4
530                         Hillingdon    WEST DRAYTON        UB7

[531 rows x 3 columns]

In [43]:
df_london['borough'] = df_london['borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df_london

borough            town  post_code
0                Bexley, Greenwich           LONDON        SE2
1    Ealing, Hammersmith and Fulham          LONDON     W3, W4
2                           Croydon         CROYDON        CR0
3                           Croydon         CROYDON        CR0
4                            Bexley  BEXLEY, SIDCUP  DA5, DA14
..                              ...             ...        ...
526                       Greenwich          LONDON       SE18
527    Sutton, Kingston upon Thames  WORCESTER PARK        KT4
528          Hammersmith and Fulham          LONDON        W12
529                      Hillingdon           HAYES        UB4
530                      Hillingdon    WEST DRAYTON        UB7

[531 rows x 3 columns]

### Feature Engineering
We can only focusing on the neighbourhoods of London, so performing the changes

In [44]:
df_london = df_london[df_london['town'].str.contains('LONDON')]
df_london

borough                    town post_code
0                Bexley, Greenwich                   LONDON       SE2
1    Ealing, Hammersmith and Fulham                  LONDON    W3, W4
6                              City                  LONDON       EC3
7                       Westminster                  LONDON       WC2
9                           Bromley                  LONDON      SE20
..                              ...                     ...       ...
521                       Redbridge                  LONDON  IG8, E18
522       Redbridge, Waltham Forest  LONDON, WOODFORD GREEN       IG8
525                          Barnet                  LONDON       N12
526                       Greenwich                  LONDON      SE18
528          Hammersmith and Fulham                  LONDON       W12

[308 rows x 3 columns]

In [45]:
def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England, GBR'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [46]:
geo_coordinates_uk = df_london['post_code']    
geo_coordinates_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
521    IG8, E18
522         IG8
525         N12
526        SE18
528         W12
Name: post_code, Length: 308, dtype: object

In [47]:
coordinates_latlng_uk = geo_coordinates_uk.apply(lambda x: get_x_y_uk(x))
coordinates_latlng_uk

0       51.492450000000076,0.12127000000003818
1        51.51324000000005,-0.2674599999999714
6       51.51200000000006,-0.08057999999994081
7       51.51651000000004,-0.11967999999995982
9       51.41009000000008,-0.05682999999993399
                        ...                   
521    51.589770000000044,0.030520000000024083
522      51.50642000000005,-0.1272099999999341
525     51.615920000000074,-0.1767399999999384
526      51.48207000000008,0.07143000000002075
528      51.50645000000003,-0.2369099999999662
Name: post_code, Length: 308, dtype: object

#### Latitude
Extracting the latitude from our previously collected coordinates

In [48]:
lat_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[0])
lat_uk

0      51.492450000000076
1       51.51324000000005
6       51.51200000000006
7       51.51651000000004
9       51.41009000000008
              ...        
521    51.589770000000044
522     51.50642000000005
525    51.615920000000074
526     51.48207000000008
528     51.50645000000003
Name: post_code, Length: 308, dtype: object

#### Longitude
Extracting the Longitude from our previously collected coordinates

In [49]:
lng_uk = coordinates_latlng_uk.apply(lambda x: x.split(',')[1])
lng_uk

0       0.12127000000003818
1       -0.2674599999999714
6      -0.08057999999994081
7      -0.11967999999995982
9      -0.05682999999993399
               ...         
521    0.030520000000024083
522     -0.1272099999999341
525     -0.1767399999999384
526     0.07143000000002075
528     -0.2369099999999662
Name: post_code, Length: 308, dtype: object

In [50]:
london_merged = pd.concat([df_london,lat_uk.astype(float), lng_uk.astype(float)], axis=1)
london_merged.columns= ['borough','town','post_code','latitude','longitude']
london_merged

borough                    town post_code  \
0                Bexley, Greenwich                   LONDON       SE2   
1    Ealing, Hammersmith and Fulham                  LONDON    W3, W4   
6                              City                  LONDON       EC3   
7                       Westminster                  LONDON       WC2   
9                           Bromley                  LONDON      SE20   
..                              ...                     ...       ...   
521                       Redbridge                  LONDON  IG8, E18   
522       Redbridge, Waltham Forest  LONDON, WOODFORD GREEN       IG8   
525                          Barnet                  LONDON       N12   
526                       Greenwich                  LONDON      SE18   
528          Hammersmith and Fulham                  LONDON       W12   

     latitude  longitude  
0    51.49245    0.12127  
1    51.51324   -0.26746  
6    51.51200   -0.08058  
7    51.51651   -0.11968  
9    51.41009   -0.05683  
..        ...        ...  
521  51.58977    0.03052  
522  51.50642   -0.12721  
525  51.61592   -0.17674  
526  51.48207    0.07143  
528  51.50645   -0.23691  

[308 rows x 5 columns]

### Visualize the Map of London
To help visualize the Map of London and the neighbourhoods in London, we make use of the folium package.

In [51]:
london = geocode(address='London, England, GBR')[0]
london_lng_coords = london['location']['x']
london_lat_coords = london['location']['y']
print(london_lng_coords, london_lat_coords)

-0.1272099999999341 51.50642000000005


In [52]:
# Creating the map of London
map_London = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_London

# adding markers to map
for latitude, longitude, borough, town in zip(london_merged['latitude'], london_merged['longitude'], london_merged['borough'], london_merged['town']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_London)  
    
map_London

### Venues in London
Using our previously defined function. Let's get the neaby venues present in each neighbourhood of London

In [53]:
venues_in_London = getNearbyVenues(london_merged['borough'], 
                                   london_merged['latitude'], 
                                   london_merged['longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and Chelsea, Hammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames


In [54]:
venues_in_London.head()

Neighbourhood  Neighbourhood Latitude  Neighbourhood Longitude  \
0  Bexley, Greenwich                 51.49245                  0.12127   
1  Bexley, Greenwich                 51.49245                  0.12127   
2  Bexley, Greenwich                 51.49245                  0.12127   
3  Bexley, Greenwich                 51.49245                  0.12127   
4  Bexley, Greenwich                 51.49245                  0.12127   

                              Venue Venue Category  
0                      Lesnes Abbey  Historic Site  
1                       Sainsbury's    Supermarket  
2                              Lidl    Supermarket  
3  Abbey Wood Railway Station (ABW)  Train Station  
4                       Bean @ Work    Coffee Shop

### Grouping by Venue Categories
We need to now see how many Venue Categories are there for further processing

In [55]:
venues_in_London.groupby('Venue Category').max()

Neighbourhood  Neighbourhood Latitude  \
Venue Category                                                        
Accessories Store               Westminster                51.51651   
Adult Boutique                    Islington                51.52969   
African Restaurant              Westminster                51.52587   
American Restaurant          Waltham Forest                51.61780   
Antique Shop                    Westminster                51.51651   
...                                     ...                     ...   
Wings Joint          Hammersmith and Fulham                51.54187   
Women's Store                   Westminster                51.55457   
Xinjiang Restaurant               Southwark                51.47480   
Yoga Studio                     Westminster                51.55457   
Zoo Exhibit                          Camden                51.53354   

                     Neighbourhood Longitude                        Venue  
Venue Category                                                             
Accessories Store                   -0.11968           James Smith & Sons  
Adult Boutique                      -0.08697  Sh! Women's Erotic Emporium  
African Restaurant                  -0.08808           Red Sea Restaurant  
American Restaurant                  0.02795                  Spielburger  
Antique Shop                        -0.11968     The London Silver Vaults  
...                                      ...                          ...  
Wings Joint                         -0.19795                     Wingmans  
Women's Store                       -0.11478           Vivien of Holloway  
Xinjiang Restaurant                 -0.09313                    Silk Road  
Yoga Studio                         -0.03558                    yogahaven  
Zoo Exhibit                         -0.14606                Penguin Beach  

[305 rows x 4 columns]

We can see 307 records, just goes to show how diverse and interesting the place is.

### One Hot Encoding 
We need to Encode our venue categories to get a better result for our clustering

In [56]:
London_venue_cat = pd.get_dummies(venues_in_London[['Venue Category']], prefix="", prefix_sep="")
London_venue_cat

Accessories Store  Adult Boutique  African Restaurant  \
0                      0               0                   0   
1                      0               0                   0   
2                      0               0                   0   
3                      0               0                   0   
4                      0               0                   0   
...                  ...             ...                 ...   
10359                  0               0                   0   
10360                  0               0                   0   
10361                  0               0                   0   
10362                  0               0                   0   
10363                  0               0                   0   

       American Restaurant  Antique Shop  Arcade  Arepa Restaurant  \
0                        0             0       0                 0   
1                        0             0       0                 0   
2                        0             0       0                 0   
3                        0             0       0                 0   
4                        0             0       0                 0   
...                    ...           ...     ...               ...   
10359                    0             0       0                 0   
10360                    0             0       0                 0   
10361                    0             0       0                 0   
10362                    0             0       0                 0   
10363                    0             0       0                 0   

       Argentinian Restaurant  Art Gallery  Art Museum  ...  \
0                           0            0           0  ...   
1                           0            0           0  ...   
2                           0            0           0  ...   
3                           0            0           0  ...   
4                           0            0           0  ...   
...                       ...          ...         ...  ...   
10359                       0            0           0  ...   
10360                       0            0           0  ...   
10361                       0            0           0  ...   
10362                       0            0           0  ...   
10363                       0            0           0  ...   

       Vietnamese Restaurant  Warehouse Store  Whisky Bar  Wine Bar  \
0                          0                0           0         0   
1                          0                0           0         0   
2                          0                0           0         0   
3                          0                0           0         0   
4                          0                0           0         0   
...                      ...              ...         ...       ...   
10359                      0                0           0         0   
10360                      0                0           0         0   
10361                      0                0           0         0   
10362                      0                0           0         0   
10363                      0                0           0         0   

       Wine Shop  Wings Joint  Women's Store  Xinjiang Restaurant  \
0              0            0              0                    0   
1              0            0              0                    0   
2              0            0              0                    0   
3              0            0              0                    0   
4              0            0              0                    0   
...          ...          ...            ...                  ...   
10359          0            0              0                    0   
10360          0            0              0                    0   
10361          0            0              0                    0   
10362          0            0              0                    0   
10363          0            

In [57]:
London_venue_cat['Neighbourhood'] = venues_in_London['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [London_venue_cat.columns[-1]] + list(London_venue_cat.columns[:-1])
London_venue_cat = London_venue_cat[fixed_columns]

London_venue_cat.head()

Neighbourhood  Accessories Store  Adult Boutique  African Restaurant  \
0  Bexley, Greenwich                   0               0                   0   
1  Bexley, Greenwich                   0               0                   0   
2  Bexley, Greenwich                   0               0                   0   
3  Bexley, Greenwich                   0               0                   0   
4  Bexley, Greenwich                   0               0                   0   

   American Restaurant  Antique Shop  Arcade  Arepa Restaurant  \
0                    0             0       0                 0   
1                    0             0       0                 0   
2                    0             0       0                 0   
3                    0             0       0                 0   
4                    0             0       0                 0   

   Argentinian Restaurant  Art Gallery  ...  Vietnamese Restaurant  \
0                       0            0  ...                      0   
1                       0            0  ...                      0   
2                       0            0  ...                      0   
3                       0            0  ...                      0   
4                       0            0  ...                      0   

   Warehouse Store  Whisky Bar  Wine Bar  Wine Shop  Wings Joint  \
0                0           0         0          0            0   
1                0           0         0          0            0   
2                0           0         0          0            0   
3                0           0         0          0            0   
4                0           0         0          0            0   

   Women's Store  Xinjiang Restaurant  Yoga Studio  Zoo Exhibit  
0              0                    0            0            0  
1              0                    0            0            0  
2              0                    0            0            0  
3              0                    0            0            0  
4              0                    0            0            0  

[5 rows x 306 columns]

### Venue categories mean value
We will group the Neighbourhoods and calculate the mean venue categories value in each Neighbourhood

In [58]:
London_grouped = London_venue_cat.groupby('Neighbourhood').mean().reset_index()
London_grouped.head()

Neighbourhood  Accessories Store  Adult Boutique  \
0                 Barnet                0.0             0.0   
1  Barnet, Brent, Camden                0.0             0.0   
2                 Bexley                0.0             0.0   
3      Bexley, Greenwich                0.0             0.0   
4     Bexley, Greenwich                 0.0             0.0   

   African Restaurant  American Restaurant  Antique Shop  Arcade  \
0                 0.0              0.00188           0.0     0.0   
1                 0.0              0.00000           0.0     0.0   
2                 0.0              0.00000           0.0     0.0   
3                 0.0              0.00000           0.0     0.0   
4                 0.0              0.00000           0.0     0.0   

   Arepa Restaurant  Argentinian Restaurant  Art Gallery  ...  \
0               0.0                0.007519          0.0  ...   
1               0.0                0.000000          0.0  ...   
2               0.0                0.000000          0.0  ...   
3               0.0                0.000000          0.0  ...   
4               0.0                0.000000          0.0  ...   

   Vietnamese Restaurant  Warehouse Store  Whisky Bar  Wine Bar  Wine Shop  \
0                0.00188              0.0         0.0       0.0        0.0   
1                0.00000              0.0         0.0       0.0        0.0   
2                0.00000              0.0         0.0       0.0        0.0   
3                0.00000              0.0         0.0       0.0        0.0   
4                0.00000              0.0         0.0       0.0        0.0   

   Wings Joint  Women's Store  Xinjiang Restaurant  Yoga Studio  Zoo Exhibit  
0          0.0            0.0                  0.0          0.0          0.0  
1          0.0            0.0                  0.0          0.0          0.0  
2          0.0            0.0                  0.0          0.0          0.0  
3          0.0            0.0                  0.0          0.0          0.0  
4          0.0            0.0                  0.0          0.0          0.0  

[5 rows x 306 columns]

### Top venue categories
Getting the top venue categories in London

In [59]:
# create a new dataframe for London
neighborhoods_venues_sorted_london = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_london['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighborhoods_venues_sorted_london.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_london.head()

Neighbourhood       1st Most Common Venue 2nd Most Common Venue  \
0                 Barnet                 Coffee Shop                  Café   
1  Barnet, Brent, Camden                      Bakery              Bus Stop   
2                 Bexley                 Supermarket         Historic Site   
3      Bexley, Greenwich  Construction & Landscaping           Golf Course   
4     Bexley, Greenwich                  Supermarket              Platform   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0         Grocery Store              Bus Stop                   Pub   
1  Gym / Fitness Center        Clothing Store        Hardware Store   
2           Coffee Shop              Platform     Convenience Store   
3               Daycare              Bus Stop           Sports Club   
4     Convenience Store         Historic Site         Train Station   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0    Italian Restaurant           Supermarket              Pharmacy   
1           Supermarket           Film Studio               Exhibit   
2         Train Station              Bus Stop           Golf Course   
3         Historic Site                  Park        Massage Studio   
4           Coffee Shop           Zoo Exhibit           Event Space   

        9th Most Common Venue 10th Most Common Venue  
0          Turkish Restaurant            Pizza Place  
1          Falafel Restaurant         Farmers Market  
2  Construction & Landscaping                   Park  
3                 Film Studio                Exhibit  
4                     Exhibit     Falafel Restaurant

## Model Building

### K Means
Let's cluster the city of london to roughly 5 to make it easier to analyze. We use the K Means clustering technique to do so.

In [60]:
# set number of clusters
k_num_clusters = 5

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_london = KMeans(n_clusters=k_num_clusters, random_state=0).fit(London_grouped_clustering)
kmeans_london

KMeans(n_clusters=5, random_state=0)

### Labelling Clustered Data

In [61]:
kmeans_london.labels_

array([0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [62]:
neighborhoods_venues_sorted_london.insert(0, 'Cluster Labels', kmeans_london.labels_ +1)

In [63]:
london_data = london_merged
london_data = london_data.join(neighborhoods_venues_sorted_london.set_index('Neighbourhood'), on='borough')
london_data.head()

borough    town post_code  latitude  longitude  \
0              Bexley, Greenwich   LONDON       SE2  51.49245    0.12127   
1  Ealing, Hammersmith and Fulham  LONDON    W3, W4  51.51324   -0.26746   
6                            City  LONDON       EC3  51.51200   -0.08058   
7                     Westminster  LONDON       WC2  51.51651   -0.11968   
9                         Bromley  LONDON      SE20  51.41009   -0.05683   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0               2           Supermarket              Platform   
1               3         Grocery Store         Train Station   
6               1                 Hotel    Italian Restaurant   
7               1                 Hotel           Coffee Shop   
9               1           Supermarket                 Hotel   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0     Convenience Store         Historic Site         Train Station   
1        Breakfast Spot                  Park     Indian Restaurant   
6           Coffee Shop  Gym / Fitness Center                   Pub   
7        Sandwich Place                  Café                   Pub   
9  Fast Food Restaurant         Grocery Store     Convenience Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0           Coffee Shop           Zoo Exhibit           Event Space   
1      Department Store           Fish Market    Falafel Restaurant   
6            Restaurant          Cocktail Bar              Wine Bar   
7    Italian Restaurant             Hotel Bar               Theater   
9                  Park  Gym / Fitness Center              Bus Stop   

  9th Most Common Venue 10th Most Common Venue  
0               Exhibit     Falafel Restaurant  
1        Farmers Market   Fast Food Restaurant  
6    Falafel Restaurant         Sandwich Place  
7            Restaurant       Sushi Restaurant  
9   Fried Chicken Joint     Chinese Restaurant

In [64]:
london_data_nonan = london_data.dropna(subset=['Cluster Labels'])

### Visualizing the clustered neighbourhood
Let's plot the clusters

In [65]:
map_clusters_london = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_data_nonan['latitude'], london_data_nonan['longitude'], london_data_nonan['borough'], london_data_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.8
        ).add_to(map_clusters_london)
        
map_clusters_london

## Examining our Clusters

Cluster 1

In [66]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 1, 
                      london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

town  Cluster Labels 1st Most Common Venue  \
6                    LONDON               1                 Hotel   
7                    LONDON               1                 Hotel   
9                    LONDON               1           Supermarket   
10                   LONDON               1           Coffee Shop   
12                   LONDON               1           Coffee Shop   
..                      ...             ...                   ...   
521                  LONDON               1                  Café   
522  LONDON, WOODFORD GREEN               1                 Hotel   
525                  LONDON               1           Coffee Shop   
526                  LONDON               1                   Pub   
528                  LONDON               1                   Pub   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
6      Italian Restaurant           Coffee Shop  Gym / Fitness Center   
7             Coffee Shop        Sandwich Place                  Café   
9                   Hotel  Fast Food Restaurant         Grocery Store   
10                    Pub                  Café            Food Truck   
12                    Pub                  Café            Food Truck   
..                    ...                   ...                   ...   
521                   Pub           Coffee Shop         Grocery Store   
522                  Café                   Pub   Monument / Landmark   
525                  Café         Grocery Store              Bus Stop   
526         Grocery Store              Bus Stop     Indian Restaurant   
528                  Café           Coffee Shop         Grocery Store   

     5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6                      Pub            Restaurant          Cocktail Bar   
7                      Pub    Italian Restaurant             Hotel Bar   
9        Convenience Store                  Park  Gym / Fitness Center   
10   Vietnamese Restaurant    Italian Restaurant                  Park   
12   Vietnamese Restaurant    Italian Restaurant                  Park   
..                     ...                   ...                   ...   
521          Metro Station                  Park    Seafood Restaurant   
522                  Plaza               Theater                Garden   
525                    Pub    Italian Restaurant           Supermarket   
526            Coffee Shop     Fish & Chips Shop    Turkish Restaurant   
528                  Hotel             Gastropub                  Park   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
6                Wine Bar    Falafel Restaurant         Sandwich Place  
7                 Theater            Restaurant       Sushi Restaurant  
9                Bus Stop   Fried Chicken Joint     Chinese Restaurant  
10   Gym / Fitness Center                 Hotel           Cocktail Bar  
12   Gym / Fitness Center                 Hotel           Cocktail Bar  
..                    ...                   ...                    ...  
521           Pizza Place                   Bar                 Bakery  
522              Pharmacy                Bakery     English Restaurant  
525              Pharmacy    Turkish Restaurant            Pizza Place  
526           Golf Course           Supermarket                   Park  
528           Pizza Place       Thai Restaurant     Italian Restaurant  

[300 rows x 12 columns]

Cluster 2

In [67]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 2, 
                      london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

town  Cluster Labels 1st Most Common Venue  \
0                 LONDON               2           Supermarket   
45   BEXLEYHEATH, LONDON               2           Supermarket   
124               LONDON               2           Supermarket   
291       LONDON, SIDCUP               2           Supermarket   
505               LONDON               2           Supermarket   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Platform     Convenience Store         Historic Site   
45          Historic Site           Coffee Shop              Platform   
124         Historic Site           Coffee Shop              Platform   
291         Historic Site           Coffee Shop              Platform   
505         Historic Site           Coffee Shop              Platform   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0           Train Station           Coffee Shop           Zoo Exhibit   
45      Convenience Store         Train Station              Bus Stop   
124     Convenience Store         Train Station              Bus Stop   
291     Convenience Store         Train Station              Bus Stop   
505     Convenience Store         Train Station              Bus Stop   

    8th Most Common Venue       9th Most Common Venue 10th Most Common Venue  
0             Event Space                     Exhibit     Falafel Restaurant  
45            Golf Course  Construction & Landscaping                   Park  
124           Golf Course  Construction & Landscaping                   Park  
291           Golf Course  Construction & Landscaping                   Park  
505           Golf Course  Construction & Landscaping                   Park

Cluster 3

In [68]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 3,
                      london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

town  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
1  LONDON               3         Grocery Store         Train Station   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
1        Breakfast Spot                  Park     Indian Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
1      Department Store           Fish Market    Falafel Restaurant   

  9th Most Common Venue 10th Most Common Venue  
1        Farmers Market   Fast Food Restaurant

Cluster 4

In [69]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 4,
                      london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

town  Cluster Labels 1st Most Common Venue  \
377  HARROW, STANMOREEDGWARE, LONDON               4                Bakery   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
377                   Gym         Metro Station     Fish & Chips Shop   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
377               Exhibit    Falafel Restaurant        Farmers Market   

    8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
377  Fast Food Restaurant   Filipino Restaurant            Film Studio

Cluster 5

In [70]:
london_data_nonan.loc[london_data_nonan['Cluster Labels'] == 5,
                      london_data_nonan.columns[[1] + list(range(5, london_data_nonan.shape[1]))]]

town  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
121  LONDON               5                Bakery              Bus Stop   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
121  Gym / Fitness Center        Clothing Store        Hardware Store   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
121           Supermarket           Film Studio               Exhibit   

    9th Most Common Venue 10th Most Common Venue  
121    Falafel Restaurant         Farmers Market

# 4. Results and Discussion

Paris is relatively smaller in size geographically. There are a lot of hangout spots including many restaurants, bars, and bistro. There are public transport in Paris such as buses, bikes, boats or ferries. For leisure and sight seeing, there are a lot of plazas, trails, parks, historic sites, clothing shops, art galleries and museums. It has a wide variety of cusines and eateries including French, Thai, Cambodian, Asian, Chinese etc. Overall, Paris seems like the relaxing vacation spot with a mix of lakes, historic spots and a wide variety of cusines to try out.

The neighbourhoods of London are very multicultural. There are a lot of different cusines including Indian, Italian, Turkish and Chinese. London seems takes a step further by having a lot of restaurants, bars, juice bars, coffee shops, fish and chips shop and breakfast spots. It has a lot of shopping options such as lea markets, flower shops, fish markets, fishing stores, clothing stores. The public transport such as buses and trains. For leisure, the neighbourhoods are set up to have lots of parks, golf courses, zoo, gyms and historic sites. Overall, London offers a multicultural, diverse and certainly an entertaining experience.


# 5. Conclusion

The purpose of this project was to explore the cities of Paris and London and see how attractive it is to potential tourists and migrants. We explored both the cities based on their postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together. We could see that each of the neighbourhoods in both the cities have a wide variety of experiences to offer which is unique in it's own way.

Both Paris and London seem to offer a vacation stay or a romantic gateaway with a lot of places to explore, beautiful landscapes and a wide variety of culture. Overall, it's up to the stakeholders to decide which experience they would prefer more and which would more to their liking.